# Guardar Información en Bases de Datos Relacionales con SQLite3 en Python

Importamos la libreria SQLite3 y creamos una conexión con la base de datos.

In [53]:
import sqlite3
con = sqlite3.connect("archivos/database.db")

Creamos un cursor para acceder a la base de datos.

In [54]:
cur = con.cursor()

Creamos una tabla.

In [55]:
cur.execute("CREATE TABLE personas(nombre, id, edad)")

In [56]:
res = cur.execute("SELECT name FROM sqlite_master")
res.fetchone()

('personas',)

In [57]:
res = cur.execute("SELECT name FROM sqlite_master WHERE name='cargo'")
res.fetchone() is None

True

Ahora vamos a agregar datos a nuetra base de datos.

In [58]:
cur.execute("""
    INSERT INTO personas VALUES
        ('Jairo', 1, 23),
        ('Carlos', 2, 31)
""")
con.commit() ## El método execute deja la transaccion o proceso abierto, debemos utilizar el método commit para aplicar los cambios

In [59]:
res = cur.execute("SELECT nombre FROM personas")
res.fetchall()

[('Jairo',), ('Carlos',)]

In [60]:
data = [
    ('Alejandro', 3, 24),
    ('Camilo', 4, 17),
    ('Santiago', 5, 28)
]
cur.executemany("INSERT INTO personas VALUES(?, ?, ?)", data)
con.commit() 

In [61]:
for row in cur.execute("SELECT nombre, edad FROM personas ORDER BY edad"):
    print(row)

('Camilo', 17)
('Jairo', 23)
('Alejandro', 24)
('Santiago', 28)
('Carlos', 31)


Podemos obtener los resultados de forma descendente.

In [62]:
for row in cur.execute("SELECT nombre, edad FROM personas ORDER BY edad DESC"):
    print(row)

('Carlos', 31)
('Santiago', 28)
('Alejandro', 24)
('Jairo', 23)
('Camilo', 17)


In [63]:
con.close()

In [64]:
con = sqlite3.connect("archivos/database.db")
cur = con.cursor()
cur.execute("DROP TABLE personas")
res = cur.execute("SELECT name FROM sqlite_master")
res.fetchone()
con.close()

## Aplicación utilizando Programación Orientada a Objetos (POO)

El uso de una base de datos relacional nos permite una mayor flexibilidad a la hora de guardar información cuando estamos programando en python. Consideremos el siguiente ejemplo: somos una empresa que esta encargada de la nomina de multiples empresas, asi que creamos una aplicación donde tenemos la información de las empresas que menejamos y sus empleados.

Supongamos que tenemos dos empresas "A" y "B" cada una de estas empresas tiene una serie de empleados "a#" y "b#" respectivamente. Queremos guardar la información en una base de datos de modo que nuestra aplicación tenga memoria de que empleados pertenecen a que empresa.

In [65]:
class Empresa():

    def __init__(self, nombre, empleados = []):
        
        self.nombre = nombre
        self.empleados = empleados

class Empleado():

    def __init__(self, nombre, cargo, fecha_inicio):

        self.nombre = nombre
        self.cargo = cargo
        self.fecha_inicio = fecha_inicio

empresa_A = Empresa(
    nombre="A",
    empleados=[
        Empleado("a1", "Gerente", 2010),
        Empleado("a2", "Vendedor", 2016),
        Empleado("a3", "Practicante", 2024),
    ]
)

empresa_B = Empresa(
    nombre="B",
    empleados=[
        Empleado("b1", "CEO", 2020),
        Empleado("b2", "Programador", 2020),
    ]
)

In [66]:
print("Los empleados de la empresa %s son:" % empresa_A.nombre)
for e in empresa_A.empleados:
    print(e.nombre)
print("-----------------------------------------")
print("Los empleados de la empresa %s son:" % empresa_B.nombre)
for e in empresa_B.empleados:
    print(e.nombre)

Los empleados de la empresa A son:
a1
a2
a3
-----------------------------------------
Los empleados de la empresa B son:
b1
b2


Ahora creemos nuestra base de datos.

In [67]:
con = sqlite3.connect("archivos/database.db")
cur = con.cursor()

Crearemos dos tablas, una donde guardaremos las empresas y otra donde guardaremos los diferentes empleados.

In [68]:
cur.execute("""CREATE TABLE empresas(
                                empresa_id INTEGER PRIMARY KEY,
                                nombre TEXT)""")
cur.execute("""CREATE TABLE empleados(
                                empleado_id INTEGER PRIMARY KEY,
                                nombre TEXT, cargo TEXT,
                                fecha_inicio INTEGER,
                                empresa_id INTEGER,
                                FOREING KEY empresa_id
                                    REFERENCES empresas(empresa_id))""")

In [69]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('empresas',), ('empleados',)]


Vamos a agregar las empresas a la tabla empresas.

In [70]:
data = [
    (empresa_A.nombre),
    (empresa_B.nombre)
]

cur.executemany("INSERT INTO empresas(nombre) VALUES(?)", data)
con.commit() 

In [71]:
for row in cur.execute("SELECT empresa_id, nombre FROM empresas"):
    print(row)

(1, 'A')
(2, 'B')


Ahora vamos a agregar los empleados a la tabla empleados. En esta parte es muy importante asignar la ID de la empresa a cada empleado dependiendo de a que empresa pertence, esto con el fin de poder establecer las relaciones.

In [72]:
## Insertamos empleados de la empresa A
for e in empresa_A.empleados:
    cur.execute("INSERT INTO empleados(nombre, cargo, fecha_inicio, empresa_id) VALUES(?, ?, ?, ?)", (e.nombre, e.cargo, e.fecha_inicio, 1))

## Insertamos empleados de la empresa B
for e in empresa_B.empleados:
    cur.execute("INSERT INTO empleados(nombre, cargo, fecha_inicio, empresa_id) VALUES(?, ?, ?, ?)", (e.nombre, e.cargo, e.fecha_inicio, 2))

con.commit()

In [73]:
for row in cur.execute("SELECT empleado_id, nombre, cargo, fecha_inicio, empresa_id FROM empleados"):
    print(row)

(1, 'a1', 'Gerente', 2010, 1)
(2, 'a2', 'Vendedor', 2016, 1)
(3, 'a3', 'Practicante', 2024, 1)
(4, 'b1', 'CEO', 2020, 2)
(5, 'b2', 'Programador', 2020, 2)


Ahora supongamos que quremos ver solamente los empleados correspondientes a la empresa A.

In [74]:
for row in cur.execute("SELECT empleados.empleado_id, empleados.nombre, empleados.cargo, empleados.fecha_inicio, empleados.empresa_id FROM empleados INNER JOIN empresas ON empleados.empresa_id=empresas.empresa_id WHERE empresas.empresa_id=1"):
    print(row)

(1, 'a1', 'Gerente', 2010, 1)
(2, 'a2', 'Vendedor', 2016, 1)
(3, 'a3', 'Practicante', 2024, 1)


Ahora los empleados de la empresa B.

In [75]:
for row in cur.execute("SELECT empleados.empleado_id, empleados.nombre, empleados.cargo, empleados.fecha_inicio, empleados.empresa_id FROM empleados INNER JOIN empresas ON empleados.empresa_id=empresas.empresa_id WHERE empresas.empresa_id=2"):
    print(row)

(4, 'b1', 'CEO', 2020, 2)
(5, 'b2', 'Programador', 2020, 2)


Ahora supongamos que cerramos nuestra aplicación y una vez la volvamos a abrir queremos reestablecer los datos, es decir, volver a crear los objetos.

In [76]:
## Recreamos la empresas
empresas = []

for row in cur.execute("SELECT nombre FROM empresas"):
    empresas.append(Empresa(nombre=row[0], empleados=[]))

## Añadimos los empleados de la empresa A
for row in cur.execute("SELECT empleados.nombre, empleados.cargo, empleados.fecha_inicio FROM empleados INNER JOIN empresas ON empleados.empresa_id=empresas.empresa_id WHERE empresas.empresa_id=1"):
    empresas[0].empleados.append(Empleado(nombre=row[0], cargo=row[1], fecha_inicio=row[2]))
## Añadimos los empleados de la empresa B
for row in cur.execute("SELECT empleados.nombre, empleados.cargo, empleados.fecha_inicio FROM empleados INNER JOIN empresas ON empleados.empresa_id=empresas.empresa_id WHERE empresas.empresa_id=2"):
    empresas[1].empleados.append(Empleado(nombre=row[0], cargo=row[1], fecha_inicio=row[2]))

In [77]:
for emp in empresas:
    print("Los empleados de la empresa %s son:" % emp.nombre)
    for e in emp.empleados:
        print(e.nombre)
    print("-----------------------------------------")

Los empleados de la empresa A son:
a1
a2
a3
-----------------------------------------
Los empleados de la empresa B son:
b1
b2
-----------------------------------------


In [78]:
con = sqlite3.connect("archivos/database.db")
cur = con.cursor()
cur.execute("DROP TABLE empleados")
cur.execute("DROP TABLE empresas")
res = cur.execute("SELECT name FROM sqlite_master")
res.fetchone()
con.close()